This notebook will read the dark matter particle table for the SMDPL simulation, and downsample it for our model.

We will create two versions:
1. default one will have 10 million particles.
2. another one with 50 million particles for test.

We will use fixed seed for random so the results can be reproduced.

* There are 6 columns in the table, which are: `X Y Z VX VY VZ ID`
* There are 113241782 particles in the table

# Particle table

In [1]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

import os
import sys
import copy 
from time import time

import numpy as np 
import matplotlib.pyplot as plt
    
import matplotlib.mlab as ml
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

%config InlineBackend.figure_format='retina'
plt.rcParams['figure.dpi'] = 100.0
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12.0
plt.rc('text', usetex=True)

# Personal repo
from stellar_mass_function import compute_smf, \
    bootstrap_smf
from load_mdpl2_mock import value_added_smdpl_mock
from model_predictions import total_stellar_mass_including_satellites
    
from hsc_massive import \
    s16a_path, \
    sample_selection, \
    prepare_sed, \
    catalog_summary, \
    smhm, \
    plotting

In [2]:
# Setup environment 
envir = s16a_path.set_env(version='astro5')
#envir = s16a_path.set_env(version='kungpao')

# Folder for new UM model 
um_dir = os.path.join(envir['s16a'], 'um2/um2_new')
um_smdpl_dir = os.path.join(um_dir, 'um_smdpl')

# The gzipped particle table
um_smdpl_ptbl = os.path.join(um_smdpl_dir, 'particles_0.712400_demo.txt')

In [3]:
particle_table_dtype = [
        ("x", "float64"), ("y", "float64"), ("z", "float64") # halo position (comoving Mpc/h)
]

In [4]:
um_smdpl_particles = np.loadtxt(um_smdpl_ptbl, usecols=[0, 1, 2],
                                dtype=particle_table_dtype)

## Try pandas

In [8]:
import pandas as pd

In [35]:
um_smdpl_pchunks = pd.read_csv(um_smdpl_ptbl, usecols=[0, 1, 2], delim_whitespace=True,
                               names=['x', 'y', 'z', 'vx', 'vy', 'vz', 'id'], 
                               dtype=particle_table_dtype, index_col=False,
                               chunksize=100)
um_smdpl_pdframe = pd.concat(um_smdpl_pchunks) 
um_smdpl_precord = um_smdpl_pdframe.to_records(index=False)

In [36]:
np.random.seed(2018)
np.random.choice(um_smdpl_precord, 100, replace=False)

array([(  0.554414,  3.620944,   4.901208),
       (  1.838998,  3.358941,  11.42914 ),
       (  1.460224,  2.60776 ,  11.639965),
       (  3.435613,  2.901746,   4.131958),
       (  1.550081,  1.911004,  10.203577),
       (  7.726359,  4.966696,   7.731567),
       (  1.45306 ,  2.564067,  11.682394),
       (  3.494777,  0.367842,   4.753884),
       (  5.267225,  5.51451 ,   5.069497),
       (  7.053959,  3.248732,   9.290101),
       (  2.121845,  2.271478,   9.827484),
       (  3.347456,  0.219273,   7.149325),
       (  0.794051,  0.771552,  12.103573),
       ( 12.435402,  5.906353,   7.086499),
       (  0.900026,  0.608125,  12.219157),
       (  0.223927,  5.363194,  11.917305),
       (  5.884121,  5.328557,  11.144092),
       (  3.102147,  5.484324,   3.493193),
       (  2.233747,  2.720783,   4.073768),
       (  4.683961,  3.210879,   2.088361),
       (  0.791117,  0.932481,  12.35815 ),
       (  1.5651  ,  2.83578 ,   6.083983),
       (  0.536318,  4.736416,  